Cross in Tray Function
======================

In [ ]:
# Test notebook loading with fixed environment
project_root = dirname(dirname(@__DIR__))
include(joinpath(project_root, ".globtim", "notebook_setup.jl"))

# Test that we can access basic Globtim functionality
println("Testing CrossInTray function access...")
try 
    f = CrossInTray
    println("✅ CrossInTray function accessible: $f")
catch e
    println("❌ CrossInTray function not accessible: $e")
end

We define the center and range of the square domain of approximation.

In [ ]:
# Constants and Parameters
n = 2
a = 10
b = 1
scale_factor = a / b
f = CrossInTray # Objective function
d = 8 # Initial Degree 
SMPL = 200   
println("Number of samples: ", SMPL^2)
TR = test_input(f, 
                dim = n,
                center = [0.0, 0.0],
                GN = SMPL, 
                sample_range = scale_factor
                )

The `Constructor` function computes the coefficients of the polynomial approximant in the basis specified, with `GN^2` samples distributed following the measure with respect to which the polynomial basis is orthogonal.
The `RationalPrecision` specifies that the conversion of the polynomial's coefficients from the tensorized orthogonal basis to the standard monomial basis is carried out in high precision.

In [ ]:
pol_cheb = Constructor(TR, d, basis=:chebyshev, precision=RationalPrecision)
pol_lege = Constructor(TR, d, basis=:legendre, precision=RationalPrecision, normalized=true);

Note that with the `Legendre` polynomials, it is necessary to run with the `normalized` option to ensure that the polynomials are orthonormal with respect to the uniform measure on the domain of approximation.

In [ ]:
@polyvar(x[1:n]) # Define polynomial ring 
real_pts_cheb = solve_polynomial_system(
    x, n, d, pol_cheb.coeffs;
    basis=pol_cheb.basis,
    precision=pol_cheb.precision,
    normalized=pol_cheb.normalized,
)

real_pts_lege = solve_polynomial_system(
    x, n, d, pol_lege.coeffs;
    basis=pol_lege.basis,
    precision=pol_lege.precision,
    normalized=pol_lege.normalized)

df_cheb = process_crit_pts(real_pts_cheb, f, TR)
df_lege = process_crit_pts(real_pts_lege, f, TR);

In [ ]:
using Optim
df_cheb, df_min_cheb = analyze_critical_points(f, df_cheb, TR, tol_dist=0.00125);
df_lege, df_min_lege = analyze_critical_points(f, df_lege, TR, tol_dist=0.00125);

In [ ]:
# Load Distributions to activate CairoMakie extension
using Distributions
println("Distributions loaded - CairoMakie extension should now be active")

# Test if the function is now available
try
    println("Testing cairo_plot_polyapprox_levelset availability...")
    methods(cairo_plot_polyapprox_levelset)
    println("✅ Function available")
catch e
    println("❌ Function still not available: $e")
end

In [ ]:
# Test extension loading explicitly
println("=== Extension Loading Diagnostics ===")

# Check if extensions are loaded
using Pkg
ext_list = Pkg.Extensions.get_extensions()
println("Available extensions:")
for (ext_name, ext_mod) in ext_list
    println("  - $ext_name: $ext_mod")
end

# Check specifically for GlobtimCairoMakieExt
if haskey(ext_list, :GlobtimCairoMakieExt)
    println("✅ GlobtimCairoMakieExt extension found!")
else
    println("❌ GlobtimCairoMakieExt extension NOT found")
end

# Try to trigger extension loading by importing required dependencies
println("\n=== Attempting to trigger extension loading ===")
using CairoMakie
using Distributions
println("CairoMakie and Distributions imported")

# Check if the function is now in the namespace
if isdefined(Globtim, :cairo_plot_polyapprox_levelset)
    println("✅ cairo_plot_polyapprox_levelset now available in Globtim namespace")
else
    println("❌ cairo_plot_polyapprox_levelset still not available in Globtim namespace")
end

# Try calling methods to see if it's available
try
    println("Testing function availability via methods...")
    methods_result = methods(Globtim.cairo_plot_polyapprox_levelset)
    println("✅ Function methods available: $methods_result")
catch e
    println("❌ Function still not accessible: $e")
    println("Extension may not have loaded properly")
end

In [ ]:
# Try to force extension loading through proper mechanisms
println("=== Forcing Extension Loading ===")

# Import Globtim first
using Globtim

# Then import the trigger packages for the extension
using CairoMakie
using Distributions

# Force package to re-evaluate extensions
using Pkg
try
    # This should trigger extension loading
    Pkg.precompile()
    println("✅ Package precompilation completed")
catch e
    println("⚠️ Precompilation had issues: $e")
end

# Check if extension is now available in Base.loaded_modules
loaded_ext = filter(x -> occursin("GlobtimCairoMakieExt", string(x)), keys(Base.loaded_modules))
if !isempty(loaded_ext)
    println("✅ GlobtimCairoMakieExt found in loaded modules: $loaded_ext")
else
    println("❌ GlobtimCairoMakieExt not found in loaded modules")
end

# Final test of function availability
try
    # Try to access the function directly
    func = getfield(Globtim, :cairo_plot_polyapprox_levelset)
    println("✅ Function accessible via getfield: $func")
    println("Function methods: $(methods(func))")
catch e
    println("❌ Function not accessible via getfield: $e")
end

In [ ]:
# Generate the level set plot using the available function
fig_2 = Globtim.cairo_plot_polyapprox_levelset(pol_lege, TR, df_lege, df_min_lege, chebyshev_levels=true)
display(fig_2)

In [ ]:
display(fig_1)

Note that with the `Legendre` polynomials, it is necessary to run with the `normalized` option to ensure that the polynomials are orthonormal with respect to the uniform measure on the domain of approximation.